В представленной лабораторной работе рассмотрим первичный анализ данных, предобработка и решение задачи классификации с помощью простейшего алгоритма машинного обучения kNN на примере реальных наблюдений о 537 577 сделках, совершенных в __Черную пятницу__ в розничных магазинах. Данные содержат атрибуты, описанные в таблице:

| Название колонки | Краткое описание |
|---|---|
| User_ID |Идентификатор покупателя |
| Product_ID | Идентификатор сделки |
| Gender | Пол |
| Age | Возрастной диапазон |
| Occupation | Время совершения сделки |
| City_Category | Категория города (A,B,C) |
| Stay_In_Current_City_Years | Количество прожитых в городе лет |
| Marital_Status | Семейное положение |
| Product_Category_1 / Product_Category_2 / Product_Category_3 | Количество позиций по категориям товара |
| Purchase | Сумма покупки, руб |

# Введение в Machine Learning в Python

Машинное обучение –  класс методов искусственного интеллекта, при помощи которых система обрабатывает большое число примеров, выявляет закономерности и использует их, чтобы прогнозировать характеристики новых данных.

Имеется множество объектов (ситуаций) и множество возможных ответов (откликов, реакций). Существует некоторая зависимость между ответами и объектами, но она неизвестна. Известна только конечная совокупность прецедентов — пар «объект, ответ», называемая обучающей выборкой. На основе этих данных требуется восстановить неявную зависимость, то есть построить алгоритм, способный для любого возможного входного объекта выдать достаточно точный классифицирующий ответ.

## Работа с данными

Построим модель для предсказания возрастной категории в зависимости от суммарной стоимости покупок.

### Необходимые импорты и настройки

Первичный анализ данных будем проводить средствами пакета Pandas - один из ориентированных на данные пакетов python, который значительно упрощает импорт и анализ данных. Pandas построен на популярных библиотеках для работы с числами `numpy` и графиками `matplotlib`, что делает манипуляции с данными и визуализацию более удобными.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Чтение данных

На входе у нас есть данные, хранящиеся в файле с расширением *.csv.

Данные из CSV файла можно считать с помощью функции `read_csv`. По умолчанию предполагается, что поля разделены запятыми.

In [2]:
blackFriday_df = pd.read_csv('BlackFriday.csv')

При обработке CSV файла с помощью pandas, получается объект под названием DataFrame, который состоит из строк и столбцов.

In [3]:
type(blackFriday_df)

pandas.core.frame.DataFrame

Для вывода начальных и конечных элементов используются функции `head` и `tail` соответственно.

Посмотрите на считанные данные

In [4]:
# code here

Проанализируйте считанные данные. Для этого можно воспользоваться функцией `describe`

In [5]:
# code here

### Выбор данных

Не все исходные данные нам понадобятся в будущем, поэтому необходимо отсеять лишние

Оставим только идентификаторы покупателя, сделки, а также пол, возраст и суммарную стоимость покупок для дальнейшего анализа.

In [6]:
blackFriday_df = blackFriday_df[['User_ID', 'Product_ID', 'Gender', 'Age', 'Purchase']]

blackFriday_df.head()

,User_ID,Product_ID,Gender,Age,Purchase
0,1000001,P00069042,F,0-17,8370
1,1000001,P00248942,F,0-17,15200
2,1000001,P00087842,F,0-17,1422
3,1000001,P00085442,F,0-17,1057
4,1000002,P00285442,M,55+,7969


Оцените исходные данные с которыми нужно будет работать (столбец `Age`). Количество уникальных записей можно посмотреть с помощью функции `value_counts`

In [7]:
# code here

## Предобработка данных для обучения

Визуализируем исходные данные

Постройте гистограмму распределения суммы покупок через `plot` с параметром `kind='hist'`

In [8]:
# code here

Далее постройте диаграмму распределения возрастных категорий через `plot` с параметром `kind='bar'`

In [9]:
# code here

Данные для обучении модели классификации должны быть в числовом формате

Необходимо убедиться в том, что исходные данные имеют тип `int` или `float`

In [10]:
blackFriday_df['Age'].dtype

dtype('O')

In [11]:
blackFriday_df['Purchase'].dtype

dtype('int64')

Как можно заметить, данные о возрастной категории представляют из себя строку, поэтому необходимо воспользоваться средствами предобработки исходных данных. Для этого можно использовать `preprocessing` из библиотеки `sklearn`

In [12]:
from sklearn import preprocessing

In [13]:
# code here

## Реализация задачи классификации с помощью алгоритма kNN

    Вспомним, что нам необходимо построить модель для предсказания возрастной категории в зависимости от суммарной стоимости покупок.

kNN расшифровывается как k Nearest Neighbor или k Ближайших Соседей — это один из самых простых алгоритмов классификации. Задача классификации в машинном обучении — это задача отнесения объекта к одному из заранее определенных классов на основании его формализованных признаков.

Для классификации каждого из объектов тестовой выборки необходимо последовательно выполнить следующие операции:

* Вычислить расстояние до каждого из объектов обучающей выборки.
* Отобрать k объектов обучающей выборки, расстояние до которых минимально.
* Класс классифицируемого объекта — это класс, наиболее часто встречающийся среди k ближайших соседей.

[Подробнее про аглоритм](https://ru.wikipedia.org/wiki/Метод_k-ближайших_соседей)

### Необходимые импорты и настройки

Библиотека `sklearn` содержит в себе все необходимые для обучения и тестирования моделей методы и функции.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Обучение модели

Создайте модель, реализующую алгоритм kNN с числом соседей 3, используя инструменты библиотеки `sklearn`

In [15]:
# code here

Для обучения необходимо загрузить в модель массивы признаков (X) и классов (y). Обратите внимание на тип загружаемых данных – __array__

In [16]:
# code here

Затем, чтобы собрать тестовую и обучающую выборку из исходных данных, воспользуемся удобной функцией `train_test_split`, реализованной в `scikit-learn`.

Размер тренировочной и тестовой выборок общепринято распределяется в отношении __70% / 30%__

In [17]:
# code here

Затем вызовом функции `fit` обучите модель

In [18]:
# code here

### Проверка точности предсказания обученной модели

Используем функцию `predict` модели, для того, чтобы предсказать возрастной диапазон по данным о стоимости покупок `X_test`

In [19]:
# code here

Для проверки точности можно использовать функцию `metrics`

In [20]:
# code here

_Попробуйте самостоятельно повысить точность предсказания_

### Для самостоятельной работы

Проанализируйте данные о возрасте и физ. характеристиках молюсков, используя [датасет](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data)

| Оригинальная колонка | Название колонки |
|---|---|
| M |sex |
| x0.455 | length |
| x0.365 | diameter |
| x0.095 | height |
| x0.514 | whole_weight |
| x0.2245 | shucked_weight |
| x0.101 | viscera_weight |
| x0.15 | shell_weight |
| x15 | rings |

Попробуйте самостоятельно обучить модель предсказывать зависимости между следующими характеристиками на Ваш выбор:

* Диаметр
* Высота
* Полный вес

## Заключение

kNN — один из простейших алгоритмов классификации, поэтому на реальных задачах он зачастую оказывается неэффективным. Помимо точности классификации, проблемой этого классификатора является скорость классификации: если в обучающей выборке N объектов, в тестовой выборе M объектов, а размерность пространства — K, то количество операций для классификации тестовой выборки может быть оценено как O(K*M*N). И тем не менее, алгоритм работы kNN является хорошим примером для начала знакомства с Machine Learning.